In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Leo el csv de eventos

events_df = pd.read_csv("data/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

In [ ]:
events_df.head()

# Generación de features
---

In [ ]:
features_df = pd.DataFrame()

In [ ]:
features_df['person'] = events_df.person.unique()

In [ ]:
features_df.head()

## Cantidad eventos totales

In [ ]:
df = events_df.groupby('person').event.count()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

In [ ]:
features_df.head()

## Cantidad por evento

In [ ]:
df = events_df.groupby('person').event.value_counts()

In [ ]:
df = df.unstack()
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl', 'lead':'cant_l',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

In [ ]:
features_df.head()

In [ ]:
features_df.fillna(0, inplace=True)

In [ ]:
features_df.head()

## Cantidad de returnings

In [ ]:
df = events_df.groupby('person').new_vs_returning.value_counts().unstack().drop(columns=['New'])

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.head()

## Tiempo total

In [ ]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])

In [ ]:
df = (events_df.groupby('person').timestamp.agg('max') - events_df.groupby('person').timestamp.agg('min'))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df = features_df.rename(columns={'timestamp':'tiempo_total'})

In [ ]:
features_df[['person', 'tiempo_total']].head()

In [ ]:
features_df.tiempo_total = pd.to_numeric(features_df.tiempo_total, downcast='float')
features_df.tiempo_total.head()

## Tiempo entre new y returning

In [ ]:
df_news = events_df.loc[events_df.new_vs_returning == 'New']
df_news[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df_returnings = events_df.loc[events_df.new_vs_returning == 'Returning']
df_returnings[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df = (df_returnings.groupby('person').timestamp.min() - df_news.groupby('person').timestamp.min())
df = df.fillna(0)
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df = features_df.rename(columns={'timestamp':'tiempo_new_returning'})

In [ ]:
features_df[['person', 'tiempo_new_returning']].head()

In [ ]:
features_df.tiempo_new_returning = pd.to_numeric(features_df.tiempo_new_returning, downcast='float')
features_df.tiempo_new_returning.head()

## Mean entre tiempos de returnings

In [ ]:
df_returnings = events_df.loc[events_df.new_vs_returning == 'Returning']
df_returnings[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df_returnings.groupby('person').head()

In [ ]:
events_df.columns

## Channel más frecuente

In [ ]:
def agg_mas_frecuente(x):
    return (x.value_counts().index[0] if(len(x.value_counts())>0) else np.nan)

In [ ]:
df = events_df.groupby('person').channel.agg(lambda x: agg_mas_frecuente(x))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.columns

In [ ]:
features_df.rename(columns={'channel':'channel_frecuente'}, inplace=True)

In [ ]:
df.value_counts().idxmax()

In [ ]:
features_df.channel_frecuente.fillna(df.value_counts().idxmax(), inplace=True)

In [ ]:
features_df.channel_frecuente.isna().value_counts()

In [ ]:
features_df[['person', 'channel_frecuente']].head()

## Device más frecuente

In [414]:
events_df.device_type.value_counts()

Smartphone    103502
Computer       97485
Tablet          2799
Unknown          283
Name: device_type, dtype: int64

In [415]:
df = events_df.groupby('person').device_type.agg(lambda x: agg_mas_frecuente(x))
df.head()

person
0008ed71      Computer
00091926      Computer
00091a7a    Smartphone
000ba417      Computer
000c79fe    Smartphone
Name: device_type, dtype: object

In [416]:
features_df = features_df.join(df, on='person')

In [417]:
features_df.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'channel_frecuente', 'device_type'],
      dtype='object')

In [418]:
features_df.rename(columns={'device_type':'device_frecuente'}, inplace=True)

In [419]:
df.value_counts().idxmax()

'Smartphone'

In [420]:
features_df.channel_frecuente.fillna(df.value_counts().idxmax(), inplace=True)

In [422]:
features_df[['person', 'device_frecuente']].head()

,person,device_frecuente
0,4886f805,Smartphone
1,ad93850f,Smartphone
2,0297fc1e,Smartphone
3,2d681dd8,Computer
4,cccea85e,Computer


# Predicciones
---

In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Separación de datos

In [ ]:
labels_df = pd.read_csv("data/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [481]:
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 17)

In [482]:
labels = train_df.label
train_df = train_df.drop(columns=['label'])

In [483]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 16)

In [484]:
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [485]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)
y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)
to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## Encoders

In [463]:
import category_encoders as ce

### Channel frecuente

In [495]:
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [496]:
X_train.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(X_train.channel_frecuente), y_train.label)

### Device frecuente

In [488]:
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [489]:
X_train.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(X_train.device_frecuente), y_train.label)

## DMatrix

In [497]:
data_dmatrix = xgb.DMatrix(data=X_train , label=labels)

## Modelo

In [498]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic',
                          base_score= 0.1, 
                          colsample_bylevel= 0.9,
                          booster='dart', 
                          colsample_bytree = 0.5,
                          learning_rate = 0.2,
                          max_depth = 8, alpha = 20,
                          gamma=10, 
                          n_estimators = 35,
                          eval_metric='auc')

In [499]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

## Tests

In [500]:
X_test.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(X_test.channel_frecuente))

In [493]:
X_test.device_frecuente = te_device_frecuente.transform(pd.DataFrame(X_test.device_frecuente))

In [501]:
preds = xg_reg.predict(X_test)

In [502]:
features_df.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'channel_frecuente',
       'device_frecuente'],
      dtype='object')

In [503]:
roc_auc_score(y_test,preds)
# Con tiempos: 0.8082572420432335
# Sin tiempos: 0.80990061942686
# Sin tiempo total (con tiempo new returning): 0.8136200978247129
# Sin tiempo new returning (con tiempo total): 0.8137392183747094

0.8152853544930327

## Predicciones posta

In [504]:
to_predict.drop(columns='person').columns

Index(['cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch', 'cant_cv',
       'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp', 'cant_vp',
       'cant_vs', 'Returning', 'channel_frecuente', 'device_frecuente'],
      dtype='object')

In [506]:
to_predict.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(to_predict.drop(columns='person').channel_frecuente))

In [508]:
to_predict.device_frecuente = te_device_frecuente.transform(pd.DataFrame(to_predict.drop(columns='person').device_frecuente))

In [509]:
preds_posta = xg_reg.predict(to_predict.drop(columns=['person']))

In [510]:
preds_posta

array([0.00844927, 0.06620744, 0.01537462, ..., 0.06890947, 0.06890947,
       0.06890947], dtype=float32)

## Generación del csv a publicar

In [511]:
to_publish = pd.DataFrame()

In [512]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [513]:
to_publish['label'] = preds_posta
to_publish.shape

(19415, 2)

In [514]:
to_publish.head()

,person,label
0,4886f805,0.008449
1,0297fc1e,0.066207
2,2d681dd8,0.015375
3,cccea85e,0.067008
4,4c8a8b93,0.047558


In [515]:
to_publish.to_csv('27_11_4.csv', index=False)

# Algo de cross validation
---

In [ ]:
params = {'objective':'binary:logistic', 'base_score':'0.3', 
                      'colsample_bylevel':'0.7', 'booster':'dart', 
                      'colsample_bytree':'0.4', 'learning_rate':'0.25',
                      'max_depth':'13', 'alpha':'25', 'gamma':'10', 
                      'n_estimators':'23', 'eval_metric':'auc'}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123)

In [ ]:
cv_results.head()

In [ ]:
import matplotlib as plt

xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()